In [ ]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official


     |████████████████████████████████| 4.4 MB 5.5 MB/s 
     |████████████████████████████████| 1.8 MB 4.2 MB/s 
     |████████████████████████████████| 90 kB 3.8 MB/s 
     |████████████████████████████████| 1.2 MB 21.4 MB/s 
     |████████████████████████████████| 636 kB 12.5 MB/s 
     |████████████████████████████████| 99 kB 7.5 MB/s 
     |████████████████████████████████| 211 kB 10.1 MB/s 
     |████████████████████████████████| 1.1 MB 30.4 MB/s 
     |████████████████████████████████| 352 kB 37.8 MB/s 
     |████████████████████████████████| 37.1 MB 46 kB/s 
     |████████████████████████████████| 43 kB 1.3 MB/s 


In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_IN = pd.read_csv('train.csv')

In [ ]:
df_IN.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
# def df_to_dataset(dataframe, shuffle=True, batch_size=32):
#   dataframe = dataframe.copy()
#   labels = dataframe.pop('target')
#   ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
#   if shuffle:
#     ds = ds.shuffle(buffer_size=len(dataframe))
#   ds = ds.batch(batch_size)
#   ds = ds.prefetch(batch_size)
#   return ds

In [ ]:
# df_raw = tf.data.Dataset.from_tensor_slices((df_IN['text'], df_IN['target']))
# df_CSV = tf.keras.utils.read_file("train.csv")
# df_raw = tf.data.experimental.make_csv_dataset(
#       'train.csv',
#       batch_size=32,
#       label_name='target',
#       select_columns = ['text', 'target'],

#      )

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_IN['text'], df_IN['target'], test_size=0.2, random_state=42)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

train_ds = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_ds = test_dataset.batch(BATCH_SIZE)

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1', trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
from tensorflow import keras


In [ ]:
class binary_classifier(tf.keras.Model):
    def __init__(self, units=32, input_dim=32):
        super(binary_classifier, self).__init__()

        self.preprocessor_link = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
        self.encoder_link = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
        
        # self.input_layer = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
        self.preprocessing_layer = hub.KerasLayer(self.preprocessor_link, name='preprocessing')
        self.encoder = hub.KerasLayer(self.encoder_link, trainable=True, name='BERT_encoder')
        self.dropout_layer = tf.keras.layers.Dropout(0.1)
        self.dense_layer = tf.keras.layers.Dense(1, activation=None, name='classifier')

    def call(self, inputs):

        out = self.preprocessing_layer(inputs)
        # out = self.encoder(out)
        # out = self.dropout_layer(out['pooled_output'])
        # out = self.dense_layer(out)
        return out

In [ ]:
bc = binary_classifier()


In [ ]:
bc.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
bc(train_ds)

ValueError: ignored

In [ ]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'pooled_output': (N 28763649    preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
______________________________________________________________________________________________

In [ ]:
classifier_model = build_classifier_model()

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

epsi = 3e-5
optimizer = optimization.create_optimizer(init_lr=epsi,
                                          num_warmup_steps=num_warmup_steps,
                                          num_train_steps=num_train_steps,
                                          optimizer_type='adamw')

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
# print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(train_ds,
                               validation_data=val_ds,
                               epochs=epochs)

Epoch 1/5
191/191 [==============================] - 100s 456ms/step - loss: 0.5389 - binary_accuracy: 0.7284 - val_loss: 0.4049 - val_binary_accuracy: 0.8253
Epoch 2/5
191/191 [==============================] - 87s 455ms/step - loss: 0.3886 - binary_accuracy: 0.8317 - val_loss: 0.4022 - val_binary_accuracy: 0.8372
Epoch 3/5
191/191 [==============================] - 87s 454ms/step - loss: 0.3192 - binary_accuracy: 0.8691 - val_loss: 0.4041 - val_binary_accuracy: 0.8477
Epoch 4/5
191/191 [==============================] - 86s 452ms/step - loss: 0.2753 - binary_accuracy: 0.8865 - val_loss: 0.4483 - val_binary_accuracy: 0.8267
Epoch 5/5
191/191 [==============================] - 86s 452ms/step - loss: 0.2410 - binary_accuracy: 0.9013 - val_loss: 0.4670 - val_binary_accuracy: 0.8293


In [ ]:
df_test = tf.data.experimental.make_csv_dataset(
      'test.csv',
      batch_size=32,
      select_columns = ['text'],

     )

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test_l = list(test['text'])

In [ ]:
test_l[:10]

In [ ]:
pr = classifier_model.predict(test_l)

In [ ]:
len(pr)

3263

In [ ]:
pr_sigm = tf.math.sigmoid(
    pr
)

In [ ]:
pred = lambda x: [1 if i > 0.5 else 0 for i in x]
pr_sigm

<tf.Tensor: shape=(3263, 1), dtype=float32, numpy=
array([[0.89383745],
       [0.9963296 ],
       [0.9852319 ],
       ...,
       [0.98236084],
       [0.8022641 ],
       [0.9185126 ]], dtype=float32)>

In [ ]:
pred_f = pred(pr_sigm)

In [ ]:
pred_f[:10]

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

In [ ]:
sub = pd.DataFrame()

In [ ]:
test.columns

Index(['id', 'keyword', 'location', 'text'], dtype='object')

In [ ]:
sub['id'] = test['id']

In [ ]:
sub['target'] = pred_f

In [ ]:
sub.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
sub.to_csv('submission.csv',index=False)